In [1]:
import torch
from torch import nn
import torch.autograd as autograd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from numpy import array
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.metrics import roc_auc_score, make_scorer, precision_score, recall_score, f1_score
ROC_scorer = make_scorer(roc_auc_score)
F1_scorer = make_scorer(f1_score)
Precision_scorer = make_scorer(precision_score)
Recall_scorer = make_scorer(recall_score)
from sklearn.svm import SVC

In [146]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# Define functions for model training

In [147]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [199]:
import random
import time


def set_seed(seed_value=101):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, loss_fn, train_dataloader, val_dataloader, y_val, epochs):
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    
    """Train the LSTM model."""   
    # Tracking best validation accuracy
    best_accuracy = 0
    best_AUC = 0
    
    train_losses = np.zeros(epochs)
    val_losses = np.zeros(epochs)

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Acc':^9} | {'Val Loss':^10} | {'Val Acc':^9} | {'Val F1':^9} |{'Val AUC':^9} |{'Elapsed':^9}")
    print("-"*60)

    for epoch_i in tqdm(range(epochs)):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0
        epoc_acc = 0
        
        train_loss = []
        #best_p = []

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_input_tbl, b_labels = batch
            b_labels = b_labels.view(-1, 1).float().to(device)
            b_input_ids, b_input_tbl, b_labels = b_input_ids.to(device), b_input_tbl.to(device), b_labels.to(device)
            
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_input_tbl)
            logits =logits.to(device)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            loss = loss.to(device)
            total_loss += loss.item()
            
            # Compute acc 
            acc = binary_acc(logits, b_labels)
            epoc_acc += acc.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()
            
            train_loss.append(loss.item())

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)
        avg_train_acc = epoc_acc / len(train_dataloader)
        
        
        train_loss = np.mean(train_loss)
        train_losses[epoch_i] = train_loss

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy, val_f1, val_AUC, prob_list, predict_labels = evaluate(model, val_dataloader, y_val)
            
            val_losses[epoch_i] = val_loss

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                #best_label = predict_labels
                
            if val_AUC > best_AUC:
                best_AUC = val_AUC
                #best_p = prob_list
                

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {avg_train_acc:^9.2f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} |{val_f1:^9.4f} | {val_AUC:^9.4f} | {time_elapsed:^9.2f}")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")
    print(f"Training complete! Best AUC: {best_AUC:.4f}.")
    return train_losses, val_losses, prob_list, predict_labels, best_AUC

def evaluate(model, val_dataloader, y_val):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []
    
    outputs_list = []
    y_pred_list = []
    probs_list = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_input_tbl, b_labels = batch
        b_labels = b_labels.view(-1, 1).float()
        b_input_ids, b_input_tbl, b_labels = b_input_ids.to(device), b_input_tbl.to(device), b_labels.to(device)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_input_tbl)
            logits = logits.to(device)
            
        y_val_probs = torch.sigmoid(logits)
        
        outputs_list.append(logits)
        probs_list.append(y_val_probs)
        
        y_pred_tag = torch.round(y_val_probs)
        y_pred_list.append(y_pred_tag)
            
        # Compute loss
        loss = loss_fn(logits, b_labels).to(device)
        val_loss.append(loss.item())

        # Get the predictions
        #preds = torch.argmax(logits, dim=1).flatten()
        preds = (logits > 0)

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    
    a = torch.cat(y_pred_list).cpu().detach().numpy()
    p = torch.cat(probs_list).cpu().detach().numpy()

    #Convert list of lists to list
    y_predict_list = np.ndarray.flatten(a).tolist()
    y_probs_list = np.ndarray.flatten(p).tolist()
    
    val_f1 = f1_score(y_val, y_predict_list)
    val_AUC = roc_auc_score(y_val, y_probs_list)
    

    return val_loss, val_accuracy, val_f1, val_AUC, y_probs_list, y_predict_list

# Import data

In [149]:
print(len(df_static))
print(len(df_series))

230
6900


# Data preprocessing 

In [152]:
#Extract column names from time-series dataset
feature_columns = df_series.columns.tolist()[1:]
feature_columns

['Spo2_Mean',
 'Spo2_cov',
 'Spo2_Max',
 'Spo2_Min',
 'Fio2_Mean',
 'Fio2_Max',
 'Fio2_Min',
 'Fio2_cov',
 'Hyper_percent']

## Normalized the continues varaible 

In [153]:
from sklearn.preprocessing import MinMaxScaler
feature = ["BW", 'GA']
autoscaler = MinMaxScaler()
df_static[feature] = autoscaler.fit_transform(df_static[feature])

In [157]:
df_demo = df_static.iloc[:,0:3]
con_feature_columns = df_demo.columns.tolist()[1:3]

In [160]:
df_static = df_static.sort_values('ohsu_mrn').reset_index(drop=True)

In [161]:
df_demo = df_demo.sort_values('ohsu_mrn').reset_index(drop=True)

In [165]:
# Re-save the dataset
X_time_series = df_series.drop("ohsu_mrn", axis=1)
X_static = df_demo.drop("ohsu_mrn", axis=1)
y  = df_static.drop("ohsu_mrn", axis=1)['binary_ROP']

## Reshape Time-series data

In [166]:
# Convert the DataFrame to a NumPy array
data_array = X_time_series.to_numpy()

# Reshape the NumPy array into the desired shape (230, 30, 9)
reshaped_array = data_array.reshape(230, 30, 9)

# Check the shape of the reshaped array
print(reshaped_array.shape)


(230, 30, 9)


## Generate PyTorch dataset

In [167]:
# Convert data to PyTorch tensors
sequence_data_tensor = torch.Tensor(reshaped_array).float()
static_features_tensor = torch.Tensor(X_static.to_numpy()).float()
binary_labels_tensor = torch.Tensor(y.to_numpy()).long()

# Create a TensorDataset
dataset = TensorDataset(sequence_data_tensor, static_features_tensor, binary_labels_tensor)

## Generate 5-fold Dataloaders of the multimodal dataset

In [215]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import random

random_seed = 101 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
np.random.seed(random_seed)
random.seed(random_seed)

# Create a list to store the data loaders for each fold
fold_data_loaders = []
# Create a list to store validation labels for each fold
fold_val_labels = []


num_folds = 5
batch_size = 4
stratified_kf = StratifiedKFold(n_splits=num_folds)

for fold, (train_indices, val_indices) in enumerate(stratified_kf.split(np.zeros(len(dataset)), binary_labels_tensor)):
    print(f'Fold [{fold + 1}/{num_folds}]')
    
    train_data = torch.utils.data.Subset(dataset, train_indices)
    val_data = torch.utils.data.Subset(dataset, val_indices)
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    
    # Add the data loaders to the list
    fold_data_loaders.append((train_loader, val_loader))
    
    # Extract the validation labels from your dataset or labels array
    val_labels = binary_labels_tensor[val_indices]
    fold_val_labels.append(val_labels)


Fold [1/5]
Fold [2/5]
Fold [3/5]
Fold [4/5]
Fold [5/5]


In [185]:
#Check labels for eahc folder
fold_val_labels

[tensor([1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1]),
 tensor([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0]),
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
         0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0]),
 tensor([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
         0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0])]

## Check the dimensions of data loaders

In [178]:
for X1, c, y in fold_data_loaders[0][0]:
    print("Shape of X1: ", X1.shape)
    print("Shape of c: ", c.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X1:  torch.Size([4, 30, 9])
Shape of c:  torch.Size([4, 2])
Shape of y:  torch.Size([4]) torch.int64


# Multimodal model with different Hyperparameters

In [218]:
import torch
import torch.nn as nn

class multiModal(nn.Module):
    """
    Multimodal Neural Network for combining time-series data and static features.

    Args:
        input_size_ve (int): The input size for the time-series data (number of features at each time point).
        hidden_size_ve (int): The hidden size for the LSTM layers.
        static_size (int): The size of the static features.
        num_classes (int): The number of output classes.
        n_LSTMlayers (int): The number of LSTM layers.
        n_node_layer1 (int): The number of nodes in the first fully connected layer.
        drop_p (float): The dropout probability for regularization.
        bidirectional (bool): Whether the LSTM layers are bidirectional.

    """

    def __init__(self, 
                 input_size_ve, 
                 hidden_size_ve, 
                 static_size, 
                 num_classes, 
                 n_LSTMlayers, 
                 n_node_layer1, 
                 drop_p, 
                 bidirectional=False):
        
        super(multiModal, self).__init__()

        self.input_size = input_size_ve
        self.hidden_size = hidden_size_ve
        self.num_layers = n_LSTMlayers
        self.n_node1 = n_node_layer1
        self.p = drop_p
        self.output = num_classes

        # LSTM layer for time-series data
        self.lstm1 = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            bidirectional=bidirectional,
            batch_first=True)

        # Static data size
        self.static_size = static_size

        # Fully connected layer to combine LSTM and static data
        self.out = nn.Linear(self.hidden_size + static_size, self.output)

    def forward(self, x1, x_static):
        """
        Forward pass of the multimodal neural network.

        Args:
            x1 (torch.Tensor): Input time-series data with shape (batch_size, sequence_length, input_size_ve).
            x_static (torch.Tensor): Static features with shape (batch_size, static_size).

        Returns:
            torch.Tensor: Output tensor with shape (batch_size, num_classes).

        """

        # Initialize LSTM hidden and cell states
        hidden_state1 = torch.zeros(self.num_layers, x1.size(0), self.hidden_size).to(device)
        cell_state1 = torch.zeros(self.num_layers, x1.size(0), self.hidden_size).to(device)

        # Forward propagate through the LSTM
        lstm_out, (hn1, cn1) = self.lstm1(x1, (hidden_state1, cell_state1))

        # Extract the last hidden state from the LSTM output
        lstm_out = lstm_out[:, -1, :]

        # Static data remains unchanged
        out2 = x_static

        # Concatenate the LSTM output and static data
        out = torch.cat([lstm_out, out2], dim=1)

        # Pass the concatenated data through a fully connected layer
        out = self.out(out)

        return out


## Set up the hyperparameters (just shows the optimal setting)

In [214]:
random_seed = 101 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
np.random.seed(random_seed)
random.seed(random_seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# pretrained_embedding, freeze_embedding, n_hidden, n_LSTMlayers, n_outputs

input_size_ve = 9
hidden_size_ve = 16
static_size =2  
num_classes =1
n_LSTMlayers = 1
n_node_layer1 = 16
drop_p = 0.5
bidirectional=False

print(device)

model = multiModal(input_size_ve,
                   hidden_size_ve,  
                   static_size,  
                   num_classes, 
                   n_LSTMlayers,
                   n_node_layer1,
                   drop_p,
                   bidirectional=bidirectional)

model.to(device)


cpu


multiModal(
  (lstm1): LSTM(9, 16, batch_first=True)
  (out): Linear(in_features=18, out_features=1, bias=True)
)

## Train and evaluate the model with 5-fold cross-validation

In [216]:
random_seed = 101 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
np.random.seed(random_seed)
random.seed(random_seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)

"""Parameters
(model, optimizer, loss_fn, train_dataloader, val_dataloader=None, epochs=10)
1. model
2. optimizer
3. loss_fn
4. train_dataloader
5. val_dataloader
6. epochs 
"""
# Specify loss function

#BATCH_SIZE = 16
epochs = 50
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-6
#1e-6 1e-3
p_weight = torch.tensor([1.64]).to(device)

#criterion = nn.BCEWithLogitsLoss()
loss_fn = nn.BCEWithLogitsLoss(pos_weight = p_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay = WEIGHT_DECAY)

num_folds = 5

sum_AUROC = []

for i in range(num_folds):
    
    y_val = fold_val_labels[i]
    train_dataloader = fold_data_loaders[i][0]
    test_dataloader = fold_data_loaders[i][1]
    
    train_losses, val_loss, best_p, best_label, best_AUC = train(model, optimizer, loss_fn, train_dataloader, test_dataloader, y_val, epochs)
    
    sum_AUROC.append(best_AUC)



cpu
Start training...

 Epoch  |  Train Loss  | Train Acc |  Val Loss  |  Val Acc  |  Val F1   | Val AUC  | Elapsed 
------------------------------------------------------------


  0%|          | 0/50 [00:00<?, ?it/s]

   1    |   0.846804   |   61.96   |  0.831987  |   64.58   | 0.0000   |  0.8519   |   0.14   
   2    |   0.844955   |   61.96   |  0.831290  |   64.58   | 0.0000   |  0.8458   |   0.13   
   3    |   0.843139   |   61.96   |  0.830603  |   64.58   | 0.0000   |  0.8418   |   0.13   
   4    |   0.840882   |   61.96   |  0.829901  |   64.58   | 0.0000   |  0.8337   |   0.13   
   5    |   0.838759   |   61.96   |  0.829155  |   62.50   | 0.0000   |  0.8296   |   0.13   
   6    |   0.836448   |   63.04   |  0.828404  |   62.50   | 0.0000   |  0.8235   |   0.13   
   7    |   0.834074   |   65.76   |  0.827599  |   66.67   | 0.2000   |  0.8235   |   0.13   
   8    |   0.831422   |   71.20   |  0.826756  |   66.67   | 0.2000   |  0.8174   |   0.13   
   9    |   0.828353   |   75.00   |  0.825805  |   75.00   | 0.5000   |  0.8073   |   0.13   
  10    |   0.825163   |   76.63   |  0.824743  |   77.08   | 0.5926   |  0.7992   |   0.13   
  11    |   0.821017   |   77.72   |  0.823613  | 

  0%|          | 0/50 [00:00<?, ?it/s]

   1    |   0.648190   |   73.37   |  0.538633  |   83.33   | 0.7778   |  0.9148   |   0.13   
   2    |   0.642797   |   73.37   |  0.539979  |   83.33   | 0.7778   |  0.9087   |   0.13   
   3    |   0.638413   |   75.00   |  0.547418  |   85.42   | 0.8000   |  0.9128   |   0.13   
   4    |   0.641257   |   72.83   |  0.550419  |   81.25   | 0.7273   |  0.9108   |   0.13   
   5    |   0.637480   |   74.46   |  0.555507  |   81.25   | 0.7273   |  0.9108   |   0.13   
   6    |   0.635536   |   73.91   |  0.560650  |   81.25   | 0.7273   |  0.9108   |   0.13   
   7    |   0.634178   |   75.54   |  0.558876  |   81.25   | 0.7273   |  0.9108   |   0.13   
   8    |   0.636106   |   72.83   |  0.560683  |   81.25   | 0.7273   |  0.9108   |   0.13   
   9    |   0.633687   |   73.91   |  0.560739  |   81.25   | 0.7273   |  0.9108   |   0.13   
  10    |   0.632526   |   74.46   |  0.561184  |   81.25   | 0.7273   |  0.9047   |   0.13   
  11    |   0.632608   |   73.91   |  0.560847  | 

...

   1    |   0.625176   |   76.63   |  0.535754  |   79.17   | 0.7059   |  0.9087   |   0.13   
   2    |   0.626795   |   76.63   |  0.537569  |   81.25   | 0.7429   |  0.8966   |   0.13   
   3    |   0.621863   |   77.17   |  0.542841  |   81.25   | 0.7429   |  0.8966   |   0.13   
   4    |   0.623028   |   76.63   |  0.540808  |   81.25   | 0.7429   |  0.8966   |   0.13   
   5    |   0.622045   |   77.17   |  0.549181  |   81.25   | 0.7273   |  0.8986   |   0.13   
   6    |   0.620299   |   77.17   |  0.542716  |   81.25   | 0.7429   |  0.8966   |   0.13   
   7    |   0.620506   |   77.17   |  0.556028  |   81.25   | 0.7273   |  0.8945   |   0.13   
   8    |   0.621205   |   77.17   |  0.538390  |   81.25   | 0.7429   |  0.8966   |   0.13   
   9    |   0.618764   |   77.17   |  0.551011  |   81.25   | 0.7273   |  0.8945   |   0.13   
  10    |   0.619396   |   77.17   |  0.542783  |   81.25   | 0.7429   |  0.8966   |   0.14   
  11    |   0.617470   |   77.72   |  0.542187  | 

  0%|          | 0/50 [00:00<?, ?it/s]

   1    |   0.578014   |   78.80   |  0.702460  |   72.92   | 0.6667   |  0.7996   |   0.14   
   2    |   0.580788   |   76.09   |  0.699864  |   72.92   | 0.6667   |  0.8016   |   0.13   
   3    |   0.574856   |   80.43   |  0.701522  |   75.00   | 0.6857   |  0.8016   |   0.13   
   4    |   0.572765   |   77.72   |  0.705502  |   75.00   | 0.6857   |  0.8016   |   0.13   
   5    |   0.572756   |   78.80   |  0.708878  |   72.92   | 0.6667   |  0.7996   |   0.13   
   6    |   0.571633   |   80.43   |  0.709776  |   72.92   | 0.6667   |  0.8016   |   0.13   
   7    |   0.573498   |   77.17   |  0.712253  |   72.92   | 0.6667   |  0.7996   |   0.13   
   8    |   0.567634   |   80.98   |  0.711381  |   75.00   | 0.6857   |  0.8016   |   0.13   
   9    |   0.569168   |   80.98   |  0.713356  |   75.00   | 0.6857   |  0.8016   |   0.13   
  10    |   0.568759   |   80.43   |  0.714518  |   72.92   | 0.6667   |  0.7996   |   0.13   
  11    |   0.568610   |   79.35   |  0.715851  | 

  0%|          | 0/50 [00:00<?, ?it/s]

   1    |   0.625709   |   75.54   |  0.427143  |   89.58   | 0.8387   |  0.9484   |   0.14   
   2    |   0.628147   |   76.63   |  0.424018  |   89.58   | 0.8387   |  0.9464   |   0.14   
   3    |   0.622123   |   75.54   |  0.420519  |   89.58   | 0.8387   |  0.9464   |   0.13   
   4    |   0.622384   |   76.09   |  0.431552  |   89.58   | 0.8387   |  0.9444   |   0.13   
   5    |   0.619587   |   75.54   |  0.420066  |   89.58   | 0.8387   |  0.9464   |   0.13   
   6    |   0.619648   |   76.09   |  0.430406  |   89.58   | 0.8387   |  0.9484   |   0.13   
   7    |   0.620144   |   75.54   |  0.430068  |   89.58   | 0.8387   |  0.9464   |   0.13   
   8    |   0.620715   |   76.63   |  0.434459  |   89.58   | 0.8387   |  0.9444   |   0.13   
   9    |   0.621162   |   75.54   |  0.429845  |   85.42   | 0.8125   |  0.9405   |   0.13   
  10    |   0.618333   |   76.09   |  0.428780  |   89.58   | 0.8387   |  0.9464   |   0.14   
  11    |   0.618253   |   75.54   |  0.434232  | 

In [217]:
np.mean(sum_AUROC)

0.8850912778904665